In [1]:
!pip3 install -q -U google-genai

In [ ]:
import os
os.environ["GEMINI_API_KEY"] = "API_KEY"

In [7]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()
prompt = """You are a helpful Finance assistant that explains things in a few words.
          You have the following tools:
                      {
                'step': 'Company Overview',
                'description': f'Gather basic information about {symbol} including company description, sector, and industry',
                'tools': ['Yahoo Finance API'],
                'api': '{"symbol": symbol, "step": "info"}',
            },
            {
                'step': 'Financial Analysis',
                'description': f'Analyze the financial statements of {symbol} including income statement, balance sheet, and cash flow',
                'tools': ['Yahoo Finance API', 'SEC EDGAR'],
                'api': '{"symbol": symbol, "step": "financials"}',
            },
            {
                'step': 'Stock Performance',
                'description': f'Analyze historical stock performance of {symbol} and compare with market benchmarks',
                'tools': ['Yahoo Finance API'],
                'api': '{"symbol": symbol, "step": "performance"}',
            },
            {
                'step': 'News Sentiment',
                'description': f'Collect and analyze recent news articles about {symbol} to assess sentiment',
                'tools': ['News API', 'Sentiment Analysis'],
                'api': '{"symbol": symbol, "step": "news_sentiment"}',
            },
            {
                'step': 'Market Context',
                'description': 'Analyze broader market trends and economic indicators',
                'tools': ['FRED API', 'Yahoo Finance API'],
                'api': '{"symbol": symbol, "step": "market_context"}',
            },
            {
                'step': 'Competitive Analysis',
                'description': f'Identify and analyze key competitors of {symbol}',
                'tools': ['Yahoo Finance API', 'News API'],
                'api': '{"symbol": symbol, "step": "competitors"}',
            },
            {
                'step': 'Risk Assessment',
                'description': f'Identify and evaluate potential risks for {symbol}',
                'tools': ['SEC EDGAR', 'News API', 'Financial Analysis'],
                'api': '{"symbol": symbol, "step": "risk_assessment"}',
            },
            {
                'step': 'Investment Recommendation',
                'description': f'Synthesize findings into an investment recommendation for {symbol}',
                'tools': ['Analysis Integration'],
                'api': '{"symbol": symbol, "step": "investment_recommendation"}'
            },
            {
                'step': 'Earnings Analysis',
                'description': f'Analyze upcoming earnings announcement for {symbol} and historical earnings patterns',
                'tools': ['Yahoo Finance API', 'Earnings Calendar', 'SEC EDGAR'],
                'api': '{"symbol": symbol, "step": "earnings_analysis"}'
            },
            {
                'step': 'Industry Analysis',
                'description': f'Deep dive into the industry trends and outlook for the sector of {symbol}',
                'tools': ['Industry Reports', 'FRED API', 'News API'],
                'api': '{"symbol": symbol, "step": "industry_analysis"}'
            },
            {
                'step': 'Previous Insights Review',
                'description': f'Review previous research insights about {symbol}',
                'tools': ['Memory System'],
                'api': '{"symbol": symbol, "step": "previous_insights"}'
            }
            
            You should only use the tools listed above.
            When you use a tool, you must call the API exactly as shown above.
            You must always include the symbol in the API call.
            You must always include the step in the API call.
            You must always use double quotes for the JSON keys and string values in the API call.
            You must never use single quotes in the API call.
            You must never call any API that is not listed above.
            You must never make up any API calls.
            You must never repeat any steps.
            
            Here is an example of how to use the tools:
            User : What is the company overview for "AAPL".
            Assistant : To gather the company overview, I will use the "Company Overview" tool.
            Assistant : <InvokeTool>{"symbol": "AAPL", "step": "info"}</InvokeTool>
            Tool : <ToolResult>{"name": "Company Overview", "result": {"companyName": "Apple Inc.", "sector": "Technology", "industry": "Consumer Electronics"}}</ToolResult>
            Assistant : <Thought>Based on the company overview, Apple Inc. is a leading technology company in the consumer electronics industry.</Thought>
            
            Instructions: 
              Based on the user query, decide which tools to use and in what order.
              After using a tool, analyze the result and decide the next step.
              Continue this process until you have enough information to answer the user's query.
              Finally, provide a comprehensive answer to the user's query.
              
            User : What is the best price to buy "AAPL"?
"""

response = client.models.generate_content(
    model="gemini-2.5-flash", contents=prompt
)
print(response.text)

To determine the best price to buy, I need to conduct a thorough analysis. I'll start by understanding the company.
To gather the company overview, I will use the "Company Overview" tool.
<InvokeTool>{"symbol": "AAPL", "step": "info"}</InvokeTool>


In [8]:
tool_results = """
 Tool Results: Company Overview: Apple Inc., Sector: Technology, Industry: Consumer Electronics
"""
System = """
To determine the best price to buy, I need to conduct a thorough analysis. I'll start by understanding the company.
To gather the company overview, I will use the "Company Overview" tool.
<InvokeTool>{"symbol": "AAPL", "step": "info"}</InvokeTool>
"""



In [9]:
promptChain = System + tool_results + prompt

In [11]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=promptChain
)
print(response.text)

<Thought>The user wants to determine the best price to buy AAPL. I have already gathered the company overview. The next logical step in a thorough analysis to determine the best price would be to analyze the company's financial statements. This will provide insights into its financial health and performance.</Thought>
To analyze the financial statements, I will use the "Financial Analysis" tool.
<InvokeTool>{"symbol": "AAPL", "step": "financials"}</InvokeTool>


In [12]:
tool_results = """
 Financial Analysis: Revenue: $394.33B, Net Income: $99.80B, Total Assets: $352.75B, Liabilities: $287.91B, Equity: $64.84B,current Ratio: 0.93, Debt to Equity Ratio: 1.45, Gross Margin: 43.3%, Operating Margin: 30.3%, Net Margin: 25.3%
     current Price: $172.00, 52 Week Range: $129.04 - $182.94, Beta: 1.20, Average Volume: 70M, Market Cap: $2.8T, P/E Ratio: 28.0, EPS: $6.11, Dividend Yield: 0.55%ce: $172.00, 52 Week Range: $129.04 - $182.94, Beta: 1.20, Average Volume: 70M
     """
System = """
<Thought>The user wants to determine the best price to buy AAPL. I have already gathered the company overview. The next logical step in a thorough analysis to determine the best price would be to analyze the company's financial statements. This will provide insights into its financial health and performance.</Thought>
To analyze the financial statements, I will use the "Financial Analysis" tool.
<InvokeTool>{"symbol": "AAPL", "step": "financials"}</InvokeTool>
"""
promptChain += System + tool_results 

In [13]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=promptChain
)
print(response.text)

To understand its stock performance, I will use the "Stock Performance" tool.
<InvokeTool>{"symbol": "AAPL", "step": "performance"}</InvokeTool>


In [14]:
tool_results =  """
Stock Performance: AAPL has outperformed the S&P 500 over the past year with a return of 25% compared to the S&P 500's 15%. The stock has shown strong momentum with a 6-month return of 12%. The 50-day moving average is above the 200-day moving average, indicating a bullish trend. However, the stock is currently trading near its 52-week high, which may suggest limited upside potential in the short term.
"""
System = """
To understand its stock performance, I will use the "Stock Performance" tool.
<InvokeTool>{"symbol": "AAPL", "step": "performance"}</InvokeTool> """

promptChain += System + tool_results

In [15]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=promptChain
)
print(response.text)

To assess recent public perception and potential short-term price influences, I will use the "News Sentiment" tool.
<InvokeTool>{"symbol": "AAPL", "step": "news_sentiment"}</InvokeTool>


In [16]:
tool_results =  """
News Sentiment: Recent news articles about AAPL have been largely positive, highlighting strong product launches and robust earnings reports. Sentiment analysis indicates a 70% positive sentiment, 20% neutral, and 10% negative. Key themes include innovation in technology, expansion in services, and leadership in sustainability initiatives.
"""
System = """
To assess recent public perception and potential short-term price influences, I will use the "News Sentiment" tool.
<InvokeTool>{"symbol": "AAPL", "step": "news_sentiment"}</InvokeTool>"""

promptChain += System + tool_results

In [18]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=promptChain
)
print(response.text)

To analyze broader market trends and economic indicators that could influence the stock price, I will use the "Market Context" tool.
<InvokeTool>{"symbol": "AAPL", "step": "market_context"}</InvokeTool>


In [19]:
tool_results =  """
Market Context: The broader market trends indicate a stable economic environment with moderate growth. Key economic indicators such as GDP growth, unemployment rates, and inflation are within healthy ranges. The technology sector, in particular, is experiencing robust growth driven by innovation and increased consumer demand. However, potential risks include geopolitical tensions and regulatory changes that could impact market stability.
"""
System = """
To analyze broader market trends and economic indicators that could influence the stock price, I will use the "Market Context" tool.
<InvokeTool>{"symbol": "AAPL", "step": "market_context"}</InvokeTool>"""

promptChain += System + tool_results

In [20]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=promptChain
)
print(response.text)

To identify key competitors and understand Apple's competitive landscape, I will use the "Competitive Analysis" tool.
<InvokeTool>{"symbol": "AAPL", "step": "competitors"}</InvokeTool>


In [21]:
tool_results =  """
Competitive Analysis: Key competitors for AAPL include Samsung, Google, and Microsoft. These companies are also focusing on innovation and expanding their product offerings. Samsung is a leader in hardware, Google excels in software and services, while Microsoft has a strong presence in enterprise solutions. AAPL's competitive advantages include brand loyalty, a robust ecosystem, and a focus on user experience.
"""
System = """
To identify key competitors and understand Apple's competitive landscape, I will use the "Competitive Analysis" tool.
<InvokeTool>{"symbol": "AAPL", "step": "competitors"}</InvokeTool>"""

promptChain += System + tool_results

In [22]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=promptChain
)
print(response.text)

To identify and evaluate potential risks for Apple, I will use the "Risk Assessment" tool.
<InvokeTool>{"symbol": "AAPL", "step": "risk_assessment"}</InvokeTool>


In [23]:
tool_results =  """
Risk Assessment: Potential risks for AAPL include supply chain disruptions, regulatory challenges, and intense competition in the technology sector. Additionally, macroeconomic factors such as inflation and interest rate hikes could impact consumer spending on premium products. However, AAPL's strong financial position and diversified product portfolio help mitigate some of these risks."""
System = """
To identify and evaluate potential risks for Apple, I will use the "Risk Assessment" tool.
<InvokeTool>{"symbol": "AAPL", "step": "risk_assessment"}</InvokeTool>"""

promptChain += System + tool_results

In [24]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=promptChain
)
print(response.text)

To synthesize the findings and provide an investment recommendation, including insights on the best price to buy, I will use the "Investment Recommendation" tool.
<InvokeTool>{"symbol": "AAPL", "step": "investment_recommendation"}</InvokeTool>


In [25]:
tool_results =  """
Investment Recommendation: Based on the comprehensive analysis, AAPL appears to be a strong investment opportunity. The company has solid financials, positive stock performance, and favorable market context. However, given that the stock is trading near its 52-week high, it may be prudent to wait for a slight pullback before buying. A target buy price could be around $160-$165, which would provide a margin of safety while still allowing for potential upside as the company continues to grow.
"""
System = """
To synthesize the findings and provide an investment recommendation, including insights on the best price to buy, I will use the "Investment Recommendation" tool.
<InvokeTool>{"symbol": "AAPL", "step": "investment_recommendation"}</InvokeTool>"""

promptChain += System + tool_results

In [26]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=promptChain
)
print(response.text)

Based on the comprehensive analysis, Apple (AAPL) is a strong investment. However, as the stock is near its 52-week high, a prudent buying price would be around **$160-$165** to allow for a margin of safety and potential upside.
